In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 24.000000000000 on 2020-12-01>

In [4]:
from django.db.models import Count

duplicates = (
            Prices.objects.values('date', 'security', 'price')
            .annotate(count_id=Count('id'))
            .filter(count_id__gt=1)
        )

total_duplicates = len(duplicates)
print((f"Found {total_duplicates} duplicate entries."))

for i, duplicate in enumerate(duplicates, start=1):
            # Keep one entry and delete the rest
            entries = Prices.objects.filter(
                date=duplicate['date'],
                security=duplicate['security'],
                price=duplicate['price']
            )
            entries_to_keep = entries.first()
            entries.exclude(id=entries_to_keep.id).delete()

            # Print status update
            print(f"Duplicates are being removed: {i} out of {total_duplicates}\r", end='', flush=True)
            
print("\nDuplicates removed.")


Found 0 duplicate entries.

Duplicates removed.


In [7]:
import datetime
from decimal import Decimal


def end_of_year_price_correction(user, year, broker_name, target_nav, asset_name):
    
    target_nav = round(Decimal(target_nav), 2)
    
    # Get the broker
    try:
        broker = Brokers.objects.get(name=broker_name)
    except Brokers.DoesNotExist:
        return {"error": f"Broker {broker_name} does not exist."}

    # Get the asset
    try:
        asset = Assets.objects.get(name=asset_name)
    except Assets.DoesNotExist:
        return {"error": f"Asset {asset_name} does not exist."}

    # Calculate end of year date
    end_of_year_date = datetime.date(year, 12, 31)

    # Fetch NAV at the end of the year
    # nav_at_end_of_year = get_nav_at_date(broker, end_of_year_date)
    nav_at_end_of_year = NAV_at_date(user.id, [broker.id], end_of_year_date, asset.currency, [])['Total NAV']
    if nav_at_end_of_year is None:
        return {"error": f"No NAV found for broker {broker_name} at the end of {year}."}

    # Fetch asset price and position at the end of the year
    price_at_end_of_year = asset.price_at_date(end_of_year_date)
    if not price_at_end_of_year:
        return {"error": f"No price found for asset {asset_name} at the end of {year}."}

    position_at_end_of_year = asset.position(end_of_year_date, [broker.id])

    # Calculate new price
    old_price = price_at_end_of_year.price
    new_price = old_price + ((target_nav - nav_at_end_of_year) / position_at_end_of_year)

    # Display information
    old_asset_value = round(Decimal(old_price * position_at_end_of_year), 2)
    new_asset_value = round(Decimal(new_price * position_at_end_of_year), 2)

    result = {
        "old_price": old_price,
        "new_price": new_price,
        "old_asset_value": old_asset_value,
        "new_asset_value": new_asset_value,
        "old_nav": nav_at_end_of_year,
        "target_nav": target_nav
    }

    print(f"Old Price: {result['old_price']}, New Price: {result['new_price']}")
    print(f"Old Asset Value: {result['old_asset_value']}, New Asset Value: {result['new_asset_value']}")
    print(f"Old NAV: {result['old_nav']}, Target NAV: {result['target_nav']}")

    # Ask for confirmation
    confirm = input("Do you want to update the price? (yes/no): ")

    if confirm.lower() == 'yes':
        # Update the price
        price_instance, created = Prices.objects.get_or_create(
            security=asset, date=end_of_year_date,
            defaults={'price': new_price}
        )
        if not created:
            price_instance.price = new_price
            price_instance.save()

        result["status"] = "Price updated successfully."
        print("New price is:", price_instance.price)
    else:
        result["status"] = "Price update canceled."

    return result

user = CustomUser.objects.filter(username='Yaroslav').first()
year = 2014
broker_name = 'UBS Pension'
target_nav = 38329.29
asset_name = 'Emerging Markets Equity Fund'

end_of_year_price_correction(user, year, broker_name, target_nav, asset_name)

utils. 89 Global Listed Property Tracker Fund 1121.78
utils. 89 UBS UK Equity Tracker 4257.81
utils. 89 UBS Pacific (exJapan) Equity Tracker 11310.71
utils. 89 UBS Europe (ex-UK) Equity Tracker 8387.25
utils. 89 Emerging Markets Equity Fund 10887.08
utils. 89 Overseas Bond Tracker Fund 2364.66
utils. 89 UBS Glo ex-UK C 0.00
utils. 89 IS PacExJap Equ 0.00
Old Price: 0.992700, New Price: 0.992700
Old Asset Value: 10887.08, New Asset Value: 10887.08
Old NAV: 38329.29, Target NAV: 38329.29


{'old_price': Decimal('0.992700'),
 'new_price': Decimal('0.992700'),
 'old_asset_value': Decimal('10887.08'),
 'new_asset_value': Decimal('10887.08'),
 'old_nav': Decimal('38329.29'),
 'target_nav': Decimal('38329.29'),
 'status': 'Price update canceled.'}

In [33]:
asset = Assets.objects.filter(ISIN="0P0000VO70").first()
print(asset.position("2015-12-31"))
print(asset.price_at_date("2015-12-31"))
print(asset.calculate_buy_in_price("2014-12-31", None, None, date(2014, 1, 1)))

def helper_calculate_buy_in_price(asset, date, currency=None, broker_id_list=None, start_date=None):
        
    is_long_position = None

    transactions = asset.transactions.filter(
        quantity__isnull=False,  # Filter out transactions where quantity is not empty
        date__lte=date
    ).values('price', 'quantity', 'date', 'currency')

    if broker_id_list is not None:
        transactions = transactions.filter(broker_id__in=broker_id_list) 
    
    if not transactions:
        return None

    # Step 2: Get latest entry date
    entry_date = asset.entry_dates(date, broker_id_list)[-1]

    if start_date and start_date > entry_date:
        entry_date = start_date

        transactions = transactions.filter(date__gte=entry_date)
        position = asset.position(entry_date, broker_id_list)
        if position != 0:
            transactions = list(transactions) + [{
                'price': asset.price_at_date(entry_date).price,
                'quantity': position,
                'date': entry_date,
                'currency': asset.currency,
            }]
            is_long_position = position > 0
    else:
        transactions = transactions.filter(date__gte=entry_date)

    if is_long_position is None:
        first_transaction = transactions.order_by('date').first()
        if first_transaction:
            is_long_position = first_transaction['quantity'] > 0

    # Step 3: Amend the calculation method. For every entry transaction buy-in price is the weighted average of previous buy-in price and price for the current transaction.
    value_entry = Decimal(0)
    quantity_entry = Decimal(0)
    previous_entry_price = Decimal(0)

    for transaction in transactions:
        
        if currency is not None:
            fx_rate = FX.get_rate(transaction['currency'], currency, transaction['date'])['FX']
        else:
            fx_rate = 1

        if fx_rate:
            current_price = transaction['price'] * fx_rate
            weight_current = transaction['quantity']

            # Calculate entry price
            previous_entry_price = value_entry / quantity_entry if quantity_entry != 0 else 0
            weight_entry_previous = quantity_entry
            # If it's a long position and the quantity is positive, or if it's a short position and the quantity is negative, use the current price. Otherwise, use the previous buy-in price.
            entry_price = current_price if (is_long_position and transaction['quantity'] > 0) or (not is_long_position and transaction['quantity'] < 0) else previous_entry_price
            
            if (weight_entry_previous + weight_current) == 0:
                entry_price = previous_entry_price
            else:
                entry_price = (previous_entry_price * weight_entry_previous + entry_price * weight_current) / (weight_entry_previous + weight_current)
            quantity_entry += transaction['quantity']
            value_entry = entry_price * quantity_entry
            
    return round(Decimal(value_entry / quantity_entry), 6) if quantity_entry else previous_entry_price

def realized_gain_loss(asset, date, currency=None, broker_id_list=None, start_date=None):
        
    realized_gain_loss_for_current_position = 0
    total_gl_before_current_position = 0
    latest_exit_date = None

    if len(asset.exit_dates(date, broker_id_list)) != 0:
        # Step 1: Find the latest date when position is 0
        latest_exit_date = asset.exit_dates(date, broker_id_list)[-1]

        # Step 2: Sum up values of all transactions before that date
        transactions_before_entry = asset.transactions.filter(date__lte=latest_exit_date, quantity__isnull=False)
        if start_date is not None:
            transactions_before_entry = transactions_before_entry.filter(date__gte=start_date)
        if broker_id_list is not None:
            transactions_before_entry = transactions_before_entry.filter(broker_id__in=broker_id_list)
        
        if currency is not None:
            for transaction in transactions_before_entry:
                fx_rate = FX.get_rate(transaction.currency, currency, transaction.date)['FX']
                if fx_rate:
                    total_gl_before_current_position -= transaction.price * transaction.quantity * fx_rate
            if start_date is not None:
                total_gl_before_current_position -= asset.price_at_date(start_date, currency).price * asset.position(start_date)
        else:
            total_gl_before_current_position = transactions_before_entry.aggregate(total=Sum(F('price') * F('quantity')))['total'] or 0
            if start_date is not None:
                total_gl_before_current_position -= asset.price_at_date(start_date).price * asset.position(start_date)

    # Step 3: Determine whether it is a long or short position
    position_at_date = asset.position(date)
    print(f'position at {date} is: {position_at_date}')

    if position_at_date != 0:
        is_long_position = position_at_date > 0
        exit_type = 'Sell' if is_long_position else 'Buy'

        # Step 4: Calculate realized gain/loss based on exit price and buy-in price
        exit_transactions = asset.transactions.filter(type=exit_type, date__lte=date)
        if latest_exit_date:
            exit_transactions = exit_transactions.filter(date__gt=latest_exit_date)
        if start_date:
            exit_transactions = exit_transactions.filter(date__gte=start_date)
        if broker_id_list is not None:
            exit_transactions = exit_transactions.filter(broker_id__in=broker_id_list)

        for exit in exit_transactions:
            print(f'Exit transaction is: {exit.type} of {exit.quantity} at {exit.date} for {exit.price}')
            buy_in_price = helper_calculate_buy_in_price(asset, exit.date, exit.currency, broker_id_list, start_date)
            print(f'Buy-in-price at {exit.date} is {buy_in_price}')
            if buy_in_price is not None:
                if currency is not None:
                    fx_rate = FX.get_rate(exit.currency, currency, exit.date)['FX']
                else:
                    fx_rate = 1
                if fx_rate:
                    realized_gain_loss_for_current_position -= (exit.price - buy_in_price) * fx_rate * (exit.quantity)
            else:
                print("WARNING: Buy-in price is not available")
                return None
            
    # print("models. line 456", self.name, realized_gain_loss_for_current_position, total_gl_before_current_position)

    return {
        "current_position": round(Decimal(realized_gain_loss_for_current_position), 2),
        "all_time": round(Decimal(total_gl_before_current_position + realized_gain_loss_for_current_position), 2)
    }

realized_gain_loss(asset, "2015-12-31", None, None, date(2015, 1, 1))



6668.565900
UBS Pacific (exJapan) Equity Tracker is at 1.197604 on 2015-12-31
1.286414
position at 2015-12-31 is: 6668.565900
Exit transaction is: Sell of -4000.537400 at 2015-07-30 for 1.210112
Buy-in-price at 2015-07-30 is 1.232403


{'current_position': Decimal('-89.18'), 'all_time': Decimal('-89.18')}